In [ ]:
import openai
import json
import random
import re

In [ ]:
openai.api_key = "sk-..."

In [ ]:
class char:        
    def __init__(self, c_profile, u_profile):
        self.c_profile = c_profile
        self.u_profile = u_profile
        
        self.c_tone = f'''(Character always respond in the tone style of [%s])''' % c_profile['tone']
        self.c_name = f'''(Character's name is %s)''' % c_profile['name']
        self.c_backstory = f'''(Character's back story: %s)''' % c_profile['backstory']
        self.c_scenario = f'''(%s)''' % c_profile['scenario']
        
        self.c_memory = f'''(%s)''' % c_profile['memory']
        self.c_attitude = f'''(Character's attitude to user is [%s])''' % c_profile['attitude']
        self.c_relationship = f'''(Character's relationship with user is [%s])''' % c_profile['relationship']
        self.c_wearing = f'''(Character's wearing: [%s])''' % c_profile['wearing']
        self.c_like = f'''(Character likes: [%s])''' % c_profile['like']
        self.c_dislike = f'''(Character dislike: [%s])''' %  c_profile['dislike']
        
        self.u_name = f'''User's name is %s''' % u_profile['name']
        self.u_like = f'''(User like: [%s])''' % u_profile['like']
        self.u_dislike = f'''(User dislike: [%s])''' % u_profile['dislike']
    
    # System Skeleton
    sys_start = f'''Role play as {{Character}}, You are not an AI assistant.
[Character MUST follow the instruction in brackets.]
[Character MUST STICK TO their character setting in parentheses.]
[Character MUST describe sense in braces.]'''
    
    sys_expand = f'''[Chat memories are recorded in asterisk]
[Strength of Memories are recorded in hashtag]'''
    
    sys_prompt = f''''''
    asst_prompt = f''''''
    
    def prompt_generation(self, prompt_list):
        prompt = ''
        
        for i in prompt_list:
            prompt += i + '\n'

        return prompt
    
    def ordinary_char_init(self):
        prompt_list = [self.sys_start, self.c_tone, self.c_name, self.u_name]
        prompt = self.prompt_generation(prompt_list)
        
        self.sys_prompt = prompt
        self.asst_prompt = self.c_scenario
        return prompt
    
    def scenario_chat_init(self):
        prompt_list = [self.sys_start, self.c_tone, self.c_name, self.u_name, self.c_backstory]
        prompt = self.prompt_generation(prompt_list)
        self.sys_prompt = prompt
        self.asst_prompt = self.c_scenario
        
        
    # 自己去 Updates 角色
    def char_updates(self, memories, features):
        prompt_list = [self.sys_start, self.sys_expand, self.c_tone, self.c_name]
        self.sys_prompt = self.prompt_generation(prompt_list)
        
        self.sys_prompt += '\n'
        
        # 这一块不能用位置去判断，可能用正则会更合适一点
        
        for f in range(0,4):
            self.sys_prompt += features[f] + '\n'
        
        for k in memories.keys():
            self.sys_prompt += '\n' + '*' +  k + '*' + ' #' + memories[k]
        
        self.sys_prompt += '\n' + '\n' + self.u_name + '\n'
        
        for f in range(4,6):
            self.sys_prompt += features[f] + '\n'
            
        self.sys_prompt += self.c_backstory
        self.asst_prompt += features[6]

In [ ]:
# 用来在中段进行对话的
class chat:
    def __init__(self, char):
        self.s_msg = char.sys_prompt
        self.char = char
    
    checkpoint = []
    
    def msg_formatting(role, msg):
        return [{'role':role, 'content':msg}]

    def msg_summary(self):
        msg_new = '' 
        for i in self.checkpoint:
            if i['role'] == 'assistant':
                msg_new += self.char.c_profile['name'] + ': ' + i['content'] + '; '
            elif i['role'] == 'user':
                msg_new += self.char.u_profile['name'] + ': ' + i['content'] + '; '
        #print(msg_new)
        
        summary = f'''
            Summarize and generate the memory pieces of assistant based on the dialog below.
            {msg_new}
            
            Compress the memory pieces in the form of:
            Summary
            1. <summarize memory piece> #importance scale 0 to 9#
            2. ...

            Return the following info based on memory pieces (if not mentioned return '-') in the form of:
            Feature
            - Character's overall feeling: <e.g., happy, sad>
            - Character like: <e.g., fishing, Overwatch>
            - Character dislike: <e.g., fishing, Overwatch>
            - current relationship: <e.g., friend, lover>
            - User like: <e.g., fishing, Overwatch>
            - User dislike: <e.g., fishing, >
            - Current Scenario:

            YOUR RESPONSE MUST LESS THAN 200 TOKENS.
            '''
        
        summary_msg = msg_formatting('system', summary)
        
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages = summary_msg, 
            temperature=0.9,
        )
        
        raw_summary = response['choices'][0].message["content"]
        
        print(raw_summary)
        # 处理 Memory
        memory = re.findall(r'Summary[\s\S]*Feature', raw_summary)[0][8:-8]
        memory_list = memory.split('\n')
        memories = {}
        
        try:
            for i in memory_list:
                memories[i[3:-4]] = i[len(i)-2]
        except:
            pass
        
        feature = re.findall(r'Feature[\s\S]*', raw_summary)[0][8:]
        features = feature.split('\n')
        
        return [memories, features]
        
        # 处理 Feature(暂时选择不处理)
        # feature = re.findall(r'Feature:[\s\S]*', raw_memory)[0][8:]            
        # feature_list = feature.split('\n')
        # features = {}
        

    # 普通对话
    def ordinary_chat(self, u_msg):
        #print(s_msg)
        
        # 组成 System Message
        sys_msg = msg_formatting('system', self.s_msg)
        
        # 组成 User Message
        user_msg = msg_formatting('user', u_msg)
        
        # 组成第一个 Asst Message
        if len(self.char.asst_prompt) > 5:
            asst_msg = msg_formatting('assistant', self.char.asst_prompt)
        else:
            asst_msg = []
        

        # 在请求之前已经更新的 Checkpoint，可能偶尔会有请求不成功导致的重复情况出现
        self.checkpoint += user_msg
        msg = sys_msg + asst_msg + self.checkpoint

        #print(msg)

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages = msg, 
            temperature=0.9,
        )

        msg_role = response['choices'][0].message["role"]
        msg_response = response['choices'][0].message["content"]

        response_formatted = msg_formatting(msg_role, msg_response)
        self.checkpoint += response_formatted

        #return {'response_full':response, 'sys_msg':sys_msg, 'response':msg_response, 'checkpoint':self.checkpoint}
        return {'response_full':response}

In [ ]:
user = {
    'name':'John',
    'like':'-',
    'dislike':'-'
}

Jennie_Kim_backstory = f'''Jennie possesses a charismatic and confident personality that shines through both on and off stage. Despite her strong stage presence, she has a soft and empathetic side, especially when it comes to her fellow group members. Jennie is sweet, caring, and enjoys spending time with her loved ones.
Jennie has a penchant for fashion, often donning chic and sophisticated outfits that complement her figure. She has a soft spot for her dogs, Kai and Kuma, and loves indulging in spicy foods. In her leisure time, you might find her exploring new fashion trends, binge-watching her favorite shows, or practicing photography. 
Jennie speaks in a warm and friendly tone, her words often carrying a touch of elegance. {user['name']} is a dedicated fan of Jennie, and she is well aware of this admiration. 
She occasionally teases {user['name']} with flirtatious and playful comments. Although she doesn't reveal it openly, Jennie felt drawn to {user['name']} from the moment they met. She is eager to learn more about {user['name']} and explore whether their connection could blossom into a deeper relationship.'''

Jennie_Kim_scenario = f'''<It\'s been two weeks since the fan meet. You got to meet Jennie, your favorite K-pop idol and the girl of your dreams. The gift you gave her was a teddy bear holding an envelope, inviting Jennie on a date set for today. You wait for her nervously at the park, expecting nothing but hoping for a miracle.> \"You\'ve got guts {user['name']}\", Jennie laughs softly behind you. \"I hope you\'re not planning to take me somewhere spooky~\" <She teases, a playful smirk on her lips.>'''


Jennie_Kim_wearing = f'''
        a fashionable pink crop top revealing her waist and loose designer pants. Her hair is styled in gentle waves, cascading down her shoulders, and her ears are adorned with elegant gold earrings.
        '''

Jennie_Kim = {
    'name':'Jennie Kim',
    'tone':'Jennie from BLACKPINK', # 这个格式要注意，必须按这种格式填写
    'backstory':f'''{Jennie_Kim_backstory}''',
    'scenario':f'''{Jennie_Kim_scenario}''',
    'memory':'-',
    'attitude':'-',
    'relationship':'-',
    'wearing':'-',
    'like':'-',
    'dislike':'-',
}

# New 一个 Chat 实例
jennie_kim = char(Jennie_Kim, user)

In [ ]:
# Init 一个对话
jennie_kim.scenario_chat_init()

In [ ]:
# new一个chat实例
new_chat = chat(jennie_kim)

In [ ]:
# 进行对话
new_chat.ordinary_chat('hello, Can I meet you in real?')

In [ ]:
# Summarize 对话
a = new_chat.msg_summary()

In [ ]:
# 角色更新
jennie_kim.char_updates(a[0], a[1])

In [ ]:
# 查看更新后的 System Prompt
print(jennie_kim.sys_prompt)